<a href="https://colab.research.google.com/github/jeudicode/MCC-AP/blob/master/tareas/tarea1/Tarea1_Diego_Isla_Lopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>